# Load Packages

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from collections import defaultdict
from datetime import datetime 
from tqdm import tqdm_notebook
from collections import Counter
from bs4 import BeautifulSoup as bs
from tqdm import tqdm_notebook
from selenium.common.exceptions import NoSuchElementException

import urllib.request
import urllib.parse
import lxml.html
import requests
import pandas as pd
import time
import math
import re

# Setting

In [2]:
insta_path = 'C:/Users/Administrator/Desktop/YJ/data/insta data/insta_data/'
user_df_path = "C:/Users/Administrator/Desktop/YJ/data/insta data/user_data/"
first_items_df_path = "C:/Users/Administrator/Desktop/YJ/data/insta data/first_item_data/"
final_items_df_path = 'C:/Users/Administrator/Desktop/YJ/data/insta data/final_item_data/'
split_file_path = 'C:/Users/Administrator/Desktop/YJ/data/insta data/user_data/split_file/'

In [3]:
now = datetime.now()
print( now )

2019-08-12 09:22:43.013260


In [2]:
# 올림 함수
def ceil(x) : 
    return math.floor(x/3 * -1) * -1

In [3]:
def save_split(data, interval, file_name) :
    start = 0
    end = 0
    loop_cnt = math.floor(data.shape[0] / interval) + 1
    for idx,_ in enumerate(tqdm_notebook(range(loop_cnt))) :
        if start + interval <= data.shape[0] :
            end = start + interval
            save_df = data.iloc[start:end,:]
            save_df.to_excel("{file_name}.{split}.xlsx".format(file_name = file_name, split = idx), index = False, encoding = "cp949")        
            start += interval

        elif start + interval > data.shape[0] :
            end = start + data.shape[0] % interval
            save_df = data.iloc[start:end,:]
            save_df.to_excel("{file_name}.{split}.xlsx".format(file_name = file_name, split = idx), index = False, encoding = "cp949")        

In [4]:
def login_by_facebookID(ID,PW) :
    global driver
    # 페리페라 url
    url = 'https://www.instagram.com/peripera_official/'
    driver.get(url)
    time.sleep(3)
    
    # 로그인 버튼 찾기
    button = driver.find_element_by_css_selector("a.tdiEy>button")
    button.click()
    time.sleep(3)
    
    # facebook id로 로그인
    btn_facebook = driver.find_element_by_css_selector("span.coreSpriteFacebookIcon.AeB99")
    btn_facebook.click()
    time.sleep(2)
    # id
    id = driver.find_element_by_css_selector("div#email_input_container>input")
    id.send_keys(ID)
    time.sleep(0.5)
    
    # password
    password = driver.find_element_by_css_selector("div._5xu4>input")
    password.send_keys(PW)
    time.sleep(1)
    
    # login
    login_btn = driver.find_element_by_css_selector("button#u_0_5")
    login_btn.click()
    
def login(ID,PW) :
    global drive
    
    # 닥터지 url
    url = 'https://www.instagram.com/clio_official/?hl=ko'
    driver.get(url)
    time.sleep(3)

    # 로그인 버튼 찾기
    button = driver.find_element_by_css_selector("a.tdiEy>button")
    button.click()
    time.sleep(3)
    
    # id
    id = driver.find_elements_by_css_selector("label.f0n8F >input")
    time.sleep(0.5)
    id[0].send_keys(ID)
    time.sleep(0.2)
    id[1].send_keys(PW)
    time.sleep(0.2)    
    btn = driver.find_element_by_css_selector("button._0mzm-.sqdOP.L3NKy")
    btn.click()

In [5]:
# driver path
path = 'C:/Users/Administrator/chromedriver'
driver = webdriver.Chrome(path)

In [6]:
# facebook id로 로그인 할 경우
#login_by_facebookID("id_name","password")
# insta id로 로그인 할 경우
login("","")

# Crawling

## 0-1 브랜드 팔로워 집단 추출

In [13]:
## 팔로워 클릭
follower = driver.find_element_by_css_selector('#react-root > section > main > div > header > section > ul > li:nth-child(2)')
follower.click()

In [14]:
## 스크롤 내리기
body = driver.find_element_by_tag_name("body")
body.click()

import time
num_of_pagedowns = 3000
while num_of_pagedowns:
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(0.5)
    num_of_pagedowns -= 1

KeyboardInterrupt: 

In [16]:
## 팔로워 id, url 추출
follower_tag = driver.find_elements_by_css_selector('a.FPmhX.notranslate._0imsa')

In [17]:
follower_list=[]
for url in follower_tag:
    follower_list.append((url.text, url.get_attribute('href')))

In [20]:
## 데이터프레임으로 변환
df_follower = pd.DataFrame(follower_list, columns=['팔로워id', '팔로워url'])
df_follower.head()

,팔로워id,팔로워url
0,pinyanat8642,https://www.instagram.com/pinyanat8642/
1,siobhansonryu,https://www.instagram.com/siobhansonryu/
2,damzz.soft,https://www.instagram.com/damzz.soft/
3,mn.dn13,https://www.instagram.com/mn.dn13/
4,saragriloo,https://www.instagram.com/saragriloo/


In [21]:
## 데이터프레임 저장
df_follower.to_excel('클리오팔로워리스트.xlsx', header=True, index=False)

#### 영제오빠 코드

In [7]:
def make_insta_df(url, crawl_time) :
    global driver
    # url
    driver.get(url)
    time.sleep(3)
    # 팔로워
    follower = driver.find_elements_by_css_selector("ul.k9GMp>li>a.-nal3")
    time.sleep(1)

    try :
        # 클릭
        follower[0].click()
    except :
        # for scroll down
        batang = driver.find_element_by_css_selector("body")
        time.sleep(0.5)
        batang.click()
        time.sleep(1.5)
        for _ in range(2) :
            batang.send_keys(Keys.PAGE_UP)
            time.sleep(1)

        # 클릭
        follower[0].click()
    
    # for scroll down
    batang = driver.find_element_by_css_selector("body")
    time.sleep(2)
    batang.click()
    time.sleep(1.5)
    
    # 시간 측정
    for_start = datetime.now()
    start_time = for_start.time()
    print("start_time :", start_time)
    
    time.sleep(0.75)
    time_list = []
    # 최대 10분동안 크롤링 하기
    while True :
        batang.send_keys(Keys.END)
        now = datetime.now()
        present_time = now.time()
        
        if present_time.minute - start_time.minute == 1 and present_time.second - start_time.second == 0 :
            tmp_user_name = driver.find_elements_by_css_selector("a.FPmhX.notranslate._0imsa ")
            time_list.append(len(tmp_user_name))
            time_list = list(set(time_list))
            
        first_condition = (present_time.minute - start_time.minute >= crawl_time and present_time.second - start_time.second == 0)
        second_condition = (start_time.hour - present_time.hour >= 1 and int(str(start_time.minute)[1]) == int(str(present_time.minute)[1]))
        if first_condition or second_condition  :
            print("finish_time :", present_time)
            break    
    print("분당 개수 :", time_list)
    try :
        # user_name 추출
        tmp_user_name = driver.find_elements_by_css_selector("a.FPmhX.notranslate._0imsa ")
        user_name = [x.text for x in tmp_user_name]

        # user_tag 추출
        tmp_user_tag = driver.find_elements_by_css_selector("div.wFPL8 ")
        user_tags = [x.text for x in tmp_user_tag]
        user_url = [x.get_attribute("href") for x in tmp_user_name]
        output_df = pd.DataFrame({"user_name" : user_name, "user_tag" : user_tags, "user_url" : user_url})

    except :
        time.sleep(1)
        # user_name 추출
        tmp_user_name = driver.find_elements_by_css_selector("a.FPmhX.notranslate._0imsa ")
        user_name = [x.text for x in tmp_user_name]

        # user_tag 추출
        tmp_user_tag = driver.find_elements_by_css_selector("div.wFPL8 ")
        user_tags = [x.text for x in tmp_user_tag]
        user_url = [x.get_attribute("href") for x in tmp_user_name]
        output_df = pd.DataFrame({"user_name" : user_name, "user_tag" : user_tags, "user_url" : user_url})

    return output_df

In [10]:
## 페리페라, 롬앤, 3CE, 구달
# url = ['https://www.instagram.com/peripera_official/','https://www.instagram.com/romandyou/','https://www.instagram.com/3ce_official/', 'https://www.instagram.com/goodal_official/']
## 더마토리, 닥터지
url = ['https://www.instagram.com/clio_official/?hl=ko']
## 해당 url을 최대 3분동안 긁어라
clio = make_insta_df(url[0],5)

start_time : 09:16:09.955389


KeyboardInterrupt: 

In [55]:
## abib를 최대 3분동안 긁어라
abib_insta = make_insta_df(url[1],3)

start_time : 13:25:46.720176
finish_time : 13:28:46.011431
분당 개수 : [1032, 1044]


In [58]:
hanyul.to_excel(insta_path + "[hanyul] insta_df_2328_071913.xlsx", index = False)
abib_insta.to_excel(insta_path + "[abib_insta] insta_df_2410_071913.xlsx", index = False)

# 페리페라 팔로워들의 정보 추출

## 1-1 팔로워들의 팔로우는?

In [11]:
hanyul_insta = pd.read_excel(insta_path + "[hanyul] insta_df_2328_071913.xlsx")

In [12]:
hanyul_insta.head()

,user_name,user_tag,user_url
0,my._.dada,NaN,https://www.instagram.com/my._.dada/
1,1997_rose.cy,NaN,https://www.instagram.com/1997_rose.cy/
2,carroll_k08,Carroll and Tec,https://www.instagram.com/carroll_k08/
3,roro87.k,NaN,https://www.instagram.com/roro87.k/
4,orzlmdabda,아름,https://www.instagram.com/orzlmdabda/


In [22]:
clio_insta = pd.read_excel('클리오팔로워리스트.xlsx')
clio_insta.head()

,팔로워id,팔로워url
0,pinyanat8642,https://www.instagram.com/pinyanat8642/
1,siobhansonryu,https://www.instagram.com/siobhansonryu/
2,damzz.soft,https://www.instagram.com/damzz.soft/
3,mn.dn13,https://www.instagram.com/mn.dn13/
4,saragriloo,https://www.instagram.com/saragriloo/


In [24]:
clio_insta = clio_insta.iloc[:648,:]
clio_insta.head()

,팔로워id,팔로워url
0,pinyanat8642,https://www.instagram.com/pinyanat8642/
1,siobhansonryu,https://www.instagram.com/siobhansonryu/
2,damzz.soft,https://www.instagram.com/damzz.soft/
3,mn.dn13,https://www.instagram.com/mn.dn13/
4,saragriloo,https://www.instagram.com/saragriloo/


In [25]:
len(clio_insta)

648

In [22]:
der_insta = pd.read_csv('더마토리팔로워리스트(2460).csv', index_col=[0])
der_insta.head()

,팔로워id,팔로워url
0,bad_sh0916,https://www.instagram.com/bad_sh0916/
1,luuphuong_anna,https://www.instagram.com/luuphuong_anna/
2,exclusive_deals_ltd,https://www.instagram.com/exclusive_deals_ltd/
3,darleen724,https://www.instagram.com/darleen724/
4,tvxq_jbj_svt_101,https://www.instagram.com/tvxq_jbj_svt_101/


In [23]:
der_insta = der_insta.iloc[:648, :]
len(der_insta)

648

In [27]:
# 에러날 경우 그대로 ctrl+enter 치면 돌아가게끔
start_idx = 0
end_idx = clio_insta.shape[0]

In [28]:
# 팔로우 하는 사람의 게시글, 팔로우, 팔로잉 숫자 크롤링
user_inform = pd.DataFrame({"user_name" : [], "items_cnts" : [],
                           "follower_cnts" : [], "following_cnts" : [],"user_url" :[]})
# 누구를 팔로우 하고 있는가?
follow_inform = pd.DataFrame({"user_name" : [], "following_name" : [], "following_tag" : [], "following_url" : []})

In [29]:
initial_idx = start_idx
for idx, url in enumerate(tqdm_notebook(clio_insta['팔로워url'].iloc[start_idx:end_idx])) :
    whole_idx = initial_idx + idx
    # 접속
    driver.get(url)
    time.sleep(3)
    
    if "비공개 계정입니다" in driver.page_source :
        start_idx += 1
        continue
        
    elif "죄송합니다. 페이지를 사용할 수 없습니다." in driver.page_source :
        start_idx += 1
        continue
    
    # 팔로우 클릭
    try :
        time.sleep(1.5)
        follow_btn = driver.find_elements_by_xpath("//ul[@class='k9GMp ']/li[3]")
        time.sleep(1.5)
        follow_btn[0].click()

    except ElementClickInterceptedException :
        # for scroll up
        batang = driver.find_element_by_css_selector("body")
        time.sleep(2)
        batang.click()
        time.sleep(1.5)
        for _ in range(10) :
            batang.send_keys(Keys.PAGE_UP)
    tmp_items = driver.find_elements_by_xpath("//ul[@class='k9GMp ']/li[1]")
    tmp_followers = driver.find_elements_by_xpath("//ul[@class='k9GMp ']/li[2]")
    
    # user inform 데이터 형성
    tmp_df = pd.DataFrame({"user_name" : clio_insta['팔로워id'].iloc[whole_idx], "items_cnts" : [tmp_items[0].text],
                           "follower_cnts" : [tmp_followers[0].text], "following_cnts" : [follow_btn[0].text], "user_url" : url})
    user_inform = user_inform.append(tmp_df)
    
    # 팔로우 수
    follow_cnt = int(re.sub("[^0-9]","",follow_btn[0].text))
    time.sleep(2)
    
    # for scroll down
    batang = driver.find_element_by_css_selector("body")
    time.sleep(2)
    batang.click()
    time.sleep(1.5)

    # 시간 측정
    for_start = datetime.now()
    start_time = for_start.time()

    time.sleep(0.75)
    time_list = []
    set_cnt = 0
    # 최대 10분동안 크롤링 하기
    while True :
        batang.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.5)
        now = datetime.now()
        present_time = now.time()

        tmp_user_name = driver.find_elements_by_css_selector("a.FPmhX.notranslate._0imsa ")
        
        # 크롤링 user_name이 변화가 없다면
        if set_cnt == len(tmp_user_name) :
            time.sleep(2)
            batang.click()
            
        if len(tmp_user_name) == follow_cnt :
            break
        elif (now - for_start).seconds >= 100 :
            break
        elif (now - for_start).seconds >= 30 and set_cnt == len(tmp_user_name) :
            break
        set_cnt += len(tmp_user_name)
        
    time.sleep(2.5)

    # user_name 추출
    tmp_user_name = driver.find_elements_by_css_selector("a.FPmhX.notranslate._0imsa ")
    user_name = [x.text for x in tmp_user_name]

    # user_tag 추출
    tmp_user_tag = driver.find_elements_by_css_selector("div.wFPL8 ")
    if len(tmp_user_tag) < 1 :
        tmp_user_tag = driver.find_elements_by_css_selector("div.PZuss div.Igw0E.IwRSH.eGOV_._4EzTm.DhRcB>div._7UhW9.xLCgt.MMzan._0PwGv.fDxYl")
    user_tags = [x.text for x in tmp_user_tag]
    user_url = [x.get_attribute("href") for x in tmp_user_name]
    try :
        output_df = pd.DataFrame({"user_name" : clio_insta['팔로워id'].iloc[whole_idx], "following_name" : user_name, "following_tag" : user_tags, "following_url" : user_url})
    except ValueError :
        start_idx += 1
        continue
        
    follow_inform = follow_inform.append(output_df)
    
    start_idx += 1
    time.sleep(2)

NameError: name 'ElementClickInterceptedException' is not defined

In [30]:
follow_inform.tail()

,user_name,following_name,following_tag,following_url
7,repray0717,_o_o_o_t_o_o_o_,ヒナタ,https://www.instagram.com/_o_o_o_t_o_o_o_/
8,repray0717,temuges,恵,https://www.instagram.com/temuges/
9,repray0717,maisondefleur_press,𝐌𝐚𝐢𝐬𝐨𝐧 𝐝𝐞 𝐅𝐋𝐄𝐔𝐑,https://www.instagram.com/maisondefleur_press/
10,repray0717,jillstuartbeauty_likes,JILL STUART Beauty 公式 リポスト専用,https://www.instagram.com/jillstuartbeauty_likes/
11,repray0717,tsukinaaaa,つきな,https://www.instagram.com/tsukinaaaa/


In [16]:
drg_insta[drg_insta['팔로워id']=='oeungyeong68']

,팔로워id,팔로워url
546,oeungyeong68,https://www.instagram.com/oeungyeong68/


In [19]:
output_df

,user_name,following_name,following_tag,following_url


In [32]:
follow_inform.to_excel('클리오_FF리스트(500).xlsx')

In [30]:
# 분할 저장
save_path = "C:\Users\Administrator\더마토리_FF크롤링"
# 저장할 데이터, 분할할 개수, 파일명
save_split(follow_inform, 10000, save_path + "hanyul follower_df_072908")